In [2]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import glob
import seaborn as sns
import folium
import os
import numpy as np
import matplotlib.colors as mcolors
%matplotlib inline

In [3]:
# Load the shapefiles
gdf1 = gpd.read_file(glob.glob("/mnt/gis-nfs/users/abreunig/superparcels/mitre/*dt30*.shp")[0])
gdf2 = gpd.read_file(glob.glob("/mnt/gis-nfs/users/abreunig/superparcels/mitre/*dt50*.shp")[0])
gdf3 = gpd.read_file(glob.glob("/mnt/gis-nfs/users/abreunig/superparcels/mitre/*dt75*.shp")[0])
gdf4 = gpd.read_file(glob.glob("/mnt/gis-nfs/users/abreunig/superparcels/mitre/*dt100*.shp")[0])
gdf5 = gpd.read_file(glob.glob("/mnt/gis-nfs/users/abreunig/superparcels/mitre/*dt150*.shp")[0])
gdf6 = gpd.read_file(glob.glob("/mnt/gis-nfs/users/abreunig/superparcels/mitre/*dt200*.shp")[0])
ref_candidates = gpd.read_file(glob.glob("/mnt/Ddrive/projects/superparcels/inputs/55107/*.shp")[0])

Spatial Join each distance threshold on itself using the OVERLAP predicate. The OVERLAP is TRUE if two polygons share some area but do not CONTAIN one another. 

In [4]:
sjoin = gpd.sjoin(gdf6, gdf6, how='left', predicate='overlaps')[['sp_id_left', 'owner_left', 'owner_right', 'geometry']]

Remove instances where the geometry overlaps itself or where the spatial join did not hit anything

In [5]:
mismatch = sjoin[sjoin['owner_left'] != sjoin['owner_right']]
mismatch = mismatch[mismatch['owner_right'].notnull()]
mismatch.head(2)


,sp_id_left,owner_left,owner_right,geometry
0,ADAMS DAWN M_0_0,ADAMS DAWN M,TRI-STATE HOLDINGS LLC,"POLYGON ((645738.217 5035624.138, 645748.499 5..."
1,ADAMS TROY G_0_0,ADAMS TROY G,REDISKE STEVEN M,"POLYGON ((633972.548 5047737.208, 633973.582 5..."


Join mismatches back to original geodataframe to obtain the *other* geometry that overlapped. There now should be the key geometry (geometry_x) and the *other* geometry (geometry_y)

In [6]:
sjoin_right = pd.merge(sjoin, gdf6[['owner', 'geometry']], left_on='owner_right', right_on='owner', how='inner')
sjoin_right.head(2)

,sp_id_left,owner_left,owner_right,geometry_x,owner,geometry_y
0,ADAMS DAWN M_0_0,ADAMS DAWN M,TRI-STATE HOLDINGS LLC,"POLYGON ((645738.217 5035624.138, 645748.499 5...",TRI-STATE HOLDINGS LLC,"POLYGON ((644500.052 5034677.576, 644478.84 50..."
1,ADAMS TROY G_0_0,ADAMS TROY G,REDISKE STEVEN M,"POLYGON ((633972.548 5047737.208, 633973.582 5...",REDISKE STEVEN M,"POLYGON ((633995.542 5046394.943, 634017.72 50..."


Create a new geometry based on the intersection of the two geometries. This geoemtry is the overlap. Calcualte the area or size.

In [7]:
sjoin_right['diff_area'] = sjoin_right.apply(lambda x: x['geometry_x'].intersection(x['geometry_y']).area, axis=1)
sjoin_right.head()


,sp_id_left,owner_left,owner_right,geometry_x,owner,geometry_y,diff_area
0,ADAMS DAWN M_0_0,ADAMS DAWN M,TRI-STATE HOLDINGS LLC,"POLYGON ((645738.217 5035624.138, 645748.499 5...",TRI-STATE HOLDINGS LLC,"POLYGON ((644500.052 5034677.576, 644478.84 50...",1.527783e-06
1,ADAMS TROY G_0_0,ADAMS TROY G,REDISKE STEVEN M,"POLYGON ((633972.548 5047737.208, 633973.582 5...",REDISKE STEVEN M,"POLYGON ((633995.542 5046394.943, 634017.72 50...",1.100238e-07
2,ADERHOLD CRAIG_0_0,ADERHOLD CRAIG,EPPLEY GORDON H,"POLYGON ((615420.898 5024433.369, 615251.507 5...",EPPLEY GORDON H,"POLYGON ((614619.993 5024859.816, 614619.993 5...",2.543494e-07
3,ADERHOLD CRAIG_0_0,ADERHOLD CRAIG,GRONSKI STEVEN M,"POLYGON ((615420.898 5024433.369, 615251.507 5...",GRONSKI STEVEN M,"POLYGON ((615405.887 5026295.879, 615409.068 5...",0.000000e+00
4,ADKINS CLAIRE A_0_0,ADKINS CLAIRE A,BOSSHARD ANDREW,"POLYGON ((654692.035 5045046.481, 654677.645 5...",BOSSHARD ANDREW,"POLYGON ((653908.833 5044641.492, 653892.815 5...",9.611155e-09


Because the overlap will be TRUE if any slight deviation in coordinates, we need to remove the insignificant overlap geometries  -- inother words any area less than ~ 1 meter.

In [8]:
overlaps = sjoin_right[sjoin_right['diff_area'] > 1]

Now that we have our final overlap dataframe, we can extract the unique intersections based on our key owner field (_left) and how many *other* owner fields by using a groupby function.

In [9]:

gb = overlaps.groupby('owner_left')['owner_right'].nunique()

Results

In [10]:
total_number_of_intersects = gb.sum()
avg_intersects = gb.mean()
total_superparcels = len(gdf6)
percent_intersects = total_number_of_intersects / total_superparcels * 100
print(f'Total number of intersects: {total_number_of_intersects}')
print(f'Total number of superparcels: {total_superparcels}')
print(f'Percent intersects: {percent_intersects:.2f}%')
print(f'Average number of intersects: {avg_intersects:.2f}')

Total number of intersects: 627
Total number of superparcels: 3538
Percent intersects: 17.72%
Average number of intersects: 1.41


In [ ]:
import numpy as np
from shapely.geometry import Polygon

def compute_mitre_limit(polygon):
    """Compute the minimum mitre limit needed to avoid truncation."""
    if isinstance(polygon, Polygon):
        coords = np.array(polygon.exterior.coords)  # Get polygon coordinates
    else: # Handle MultiPolygon
        coords = [list(p.exterior.coords) for p in polygon.geoms]
        # Flatten list of coordinates
        coords = [item for sublist in coords for item in sublist]
        
    n = len(coords) - 1  # Ignore duplicate last point
    mitre_ratios = []

    for i in range(n):
        # Get three consecutive points (previous, current, next)
        p1, p2, p3 = coords[i - 1], coords[i], coords[(i + 1) % n]

        # Compute vectors
        v1, v2 = np.array(p1) - np.array(p2), np.array(p3) - np.array(p2)

        # Compute dot product and norms
        dot_product = np.dot(v1, v2)
        norm_v1, norm_v2 = np.linalg.norm(v1), np.linalg.norm(v2)
        norm_product = norm_v1 * norm_v2

        # Skip if vectors are degenerate (i.e., points are the same)
        if norm_product == 0:
            continue

        # Compute angle θ between vectors
        cos_theta = np.clip(dot_product / norm_product, -1, 1)  # Avoid precision errors
        theta = np.arccos(cos_theta)  # Angle in radians

        # Compute mitre ratio
        if theta > 0:  # Avoid division by zero
            mitre_ratio = 1 / np.sin(theta / 2)
            mitre_ratios.append(mitre_ratio)

    # Return max mitre ratio as the required mitre limit
    return max(mitre_ratios) if mitre_ratios else 2  # Default to 2

# Example polygon (with potential duplicate points)
polygon = Polygon([(0, 0), (4, 1), (4, 1), (3, 5), (1, 4), (0, 0)])

# Compute required mitre limit
mitre_limit = compute_mitre_limit(polygon)
print(f"Suggested mitre limit: {mitre_limit:.2f}")


In [ ]:
from shapely.geometry import Polygon, MultiPolygon
# create a complex polygon
polygon = Polygon([(0, 0), (2, 0), (2, 2), (0, 2)])
triangle = Polygon([(3, 3), (4, 3), (3.5, 4)])
multi_polygon = MultiPolygon([polygon, triangle])
mitre_limit = compute_mitre_limit(multi_polygon)
print(mitre_limit)
gdf = gpd.GeoSeries([
    multi_polygon,
    multi_polygon.buffer(5, join_style=2, mitre_limit=mitre_limit)  # Mitre
])

fig, ax = plt.subplots()
gdf.boundary.plot(ax=ax, color='k')
gdf.plot(ax=ax, facecolor='lightgray', edgecolor='black')
plt.show()



In [ ]:
df = pd.read_csv('/mnt/Ddrive/projects/superparcels/outputs/dt_analysis.csv', index_col=0)
df

In [ ]:
def zfill(s, width):
    return s.zfill(width)

# length of an integer


indexes = df.index
idx_list = []
for i in indexes:
    if len(str(i)) == 4:
        i = zfill(str(i), 5)
    idx_list.append(i)
    

In [ ]:
# convert all elements to str
idx_list = [str(i) for i in idx_list]
idx_list

In [ ]:
df.columns

In [ ]:
df

In [ ]:
df.values[0]

In [ ]:
# Separate the index values and bar values
indexes = df.index.to_list()
bar_values = df.values

# Create one subplot for each row in the df
nrows = df.shape[0]
fig, axes = plt.subplots(nrows=nrows, ncols=1, figsize=(8, 3*nrows))

# In case there is only one row, ensure axes is iterable.
if nrows == 1:
    axes = [axes]

for i, ax in enumerate(axes):
    # Create the bar plot for the current row
    bars = ax.bar(df.columns.to_list(), bar_values[i])
   
    ax.bar_label(bars, padding=3)
    
    # Set the title of the subplot to show the index value
    ax.set_title(f"FIPS {indexes[i]}")
    
    # Optionally, label axes
    ax.set_xlabel("Distance Threshold")
    ax.set_ylabel("Unique Owners")

plt.tight_layout()
plt.show()
